In [38]:
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import StandardScaler

from joblib import dump, load
import pandas as pd
import numpy as np

from psyke import Extractor, Clustering, Target
from psyke.extraction.hypercubic.strategy import AdaptiveStrategy
from psyke.extraction.hypercubic import Grid, FeatureRanker
from psyke.utils.logic import pretty_theory

from plot import *

In [39]:
def plot(testB, x, pred, extracted):
    missing = pd.read_csv("data/missing.csv", parse_dates = [0], index_col = 0)
    df = pd.read_csv("data/averaged1.csv", parse_dates = [0], index_col = 0)
    df.LPFnorm[missing.index] = np.nan

    for i, b in bartels.iterrows():
        if b.n != testB:
            continue
        xminmax = [b.t0, b.t1]

        f, axes = plt.subplots(3, figsize=(10, 8)) # l x h

        myplot(axes[0], [3, 1], "LPF", df.index, df.LPFnorm, xminmax)
        myplot(axes[0], [3, 1], "", missing.index, missing.LPF0, xminmax, color='r', marker='*', lw=0, size=10)
        myplot(axes[1], [3, 2], "V", df.index, df.V, xminmax)
        myplot(axes[1], [3, 2], "", x, pred, xminmax, color='b', marker='.', lw=0)
        myplot(axes[1], [3, 2], "", x, extracted, xminmax, color='r', marker='.', lw=0)
        myplot(axes[2], [3, 3], "B", df.index, df.B, xminmax)
        plt.subplots_adjust(hspace=0.6)
        plt.savefig(f"plot/{b.n}.jpg", dpi=96 * 2)
        plt.show()

In [75]:
def splitTest(data):
    b = bartels[(bartels.n == 2490)]
    data = data[(data.index >= b.t1.values[0])]

    b = bartels[(bartels.n == 2495) | (bartels.n == 2501) | (bartels.n == 2506)]
    idx = np.zeros_like(data.index, dtype='bool')
    for _, row in b.iterrows():
        t0, t1 = row.t0, row.t1
        idx = idx | (data.index >= t0) & (data.index < t1)
    return data[~idx], data[idx]

bartels = pd.read_csv("data/bartels.csv", parse_dates = [1, 2])
data = pd.read_csv(f'data/halffuzzycoefs4B3.csv', parse_dates=[0], index_col=0)
train, test = splitTest(data)
train.to_csv('data/alltrain.csv')
test.to_csv('data/alltest.csv')

data = pd.read_csv(f'data/pruned.csv', parse_dates=[0], index_col=0)
train, test = splitTest(data)
train.to_csv('data/prunedtrain.csv')
test.to_csv('data/prunedtest.csv')

In [73]:
pd.read_csv('data/alltrain.csv', parse_dates=[0], index_col=[0])

,Bmin,Bmedian,Bmax,Btrend,Bhalf1,Bhalf2,Bthird1,Bthird2,Bthird3,GCRmin,...,GCRhalf1,GCRhalf2,GCRthird1,GCRthird2,GCRthird3,GCRfourth1,GCRfourth2,GCRfourth3,GCRfourth4,V
2016-03-04 00:00:00,4.0,7.5,12.3,-0.080751,0.009474,-0.106862,-0.001462,-0.016066,-0.068720,-3.883201,...,-0.034045,0.060357,0.002682,-0.072897,0.065471,0.007162,-0.092158,0.046471,0.069499,410.0
2016-03-04 01:00:00,4.0,7.5,12.3,-0.086067,-0.012065,-0.101174,-0.004731,-0.012913,-0.074356,-3.883201,...,-0.035872,0.061504,0.000701,-0.070397,0.064501,0.006544,-0.095190,0.049612,0.069178,400.0
2016-03-04 02:00:00,4.0,7.5,12.3,-0.090328,-0.029534,-0.094717,-0.007107,-0.008399,-0.080310,-3.883201,...,-0.038517,0.062143,-0.002516,-0.066904,0.064396,0.004833,-0.098934,0.049670,0.071374,395.0
2016-03-04 03:00:00,4.0,7.5,12.3,-0.094527,-0.046781,-0.089474,-0.010202,-0.001853,-0.085397,-3.883201,...,-0.040322,0.063626,-0.002422,-0.062109,0.066305,0.004309,-0.102389,0.052382,0.072217,408.0
2016-03-04 04:00:00,4.0,7.5,12.3,-0.098312,-0.062166,-0.087105,-0.012979,0.006375,-0.089585,-3.883201,...,-0.042501,0.064246,-0.005347,-0.056971,0.065904,0.002981,-0.105296,0.052220,0.070273,406.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-02 19:00:00,1.6,8.0,18.2,0.168533,0.139028,0.004049,-0.032341,0.005700,0.162350,-4.670171,...,-0.059403,0.016938,-0.075540,0.018295,-0.025042,-0.074034,-0.027912,0.071203,-0.054068,477.0
2017-07-02 20:00:00,1.6,8.3,18.2,0.165550,0.157085,-0.019089,-0.032566,0.005048,0.159932,-4.670171,...,-0.058878,0.015490,-0.076592,0.020250,-0.027076,-0.076765,-0.026023,0.071905,-0.053493,442.0
2017-07-02 21:00:00,1.6,8.3,18.2,0.161067,0.174393,-0.041862,-0.033405,0.002897,0.157075,-4.670171,...,-0.058062,0.014373,-0.076522,0.022473,-0.028479,-0.079294,-0.022508,0.073575,-0.054818,437.0
2017-07-02 22:00:00,1.6,8.5,18.2,0.156652,0.188077,-0.061781,-0.033445,0.001775,0.153994,-4.670171,...,-0.057460,0.013574,-0.077431,0.025158,-0.029432,-0.080745,-0.020645,0.074790,-0.054839,436.0


In [3]:
def getTrainTest(data, testB):
    b = bartels[bartels.n==testB]
    t0, t1 = b.t0.values[0], b.t1.values[0]
    idx = (data.index >= t0) & (data.index < t1)
    return data[~idx], data[idx]

def getScaler(train, name):
    scaler = StandardScaler().fit(train)
    dump(scaler, f"scalers/scalerV{name}.joblib")
    normalization = {key: (m, s) for key, m, s in zip(train.columns, scaler.mean_, scaler.scale_)}
    return scaler, pd.DataFrame(scaler.transform(train), columns=train.columns), normalization

In [4]:
def gridex(model, train, test, normalization):
    ranked = FeatureRanker(train.columns).fit(model, train.iloc[:, :-1]).rankings()
    gridEx = Extractor.gridex(model, Grid(1, AdaptiveStrategy(ranked, [(0.6, 3), (0.75, 4)])),
                              threshold=.5, min_examples=1, normalization=normalization)
    gridEx.extract(train)
    return gridEx.brute_predict(test), gridEx.n_rules, sum([p is None for p in gridEx.predict(test)])
    
def gridrex(model, train, test, normalization):
    ranked = FeatureRanker(train.columns).fit(model, train.iloc[:, :-1]).rankings()
    gridREx = Extractor.gridrex(model, Grid(1, AdaptiveStrategy(ranked, [(0.5, 3)])),
                                threshold=.5, min_examples=1, normalization=normalization)
    gridREx.extract(train)
    return gridREx.brute_predict(test, 'default'), gridREx.n_rules, sum([p is None for p in gridREx.predict(test)])

def cart(model, train, test, normalization):
    CART = Extractor.cart(model, max_depth=10, max_leaves=10, normalization=normalization)
    CART.extract(train)
    return CART.predict(test), CART.n_rules, sum([p is None for p in CART.predict(test)])

def cosmik(model, train, test, normalization):
    COSMiK = Extractor.cosmik(model, max_components=10, k=125, patience=12, close_to_center=True,
                              output=Target.REGRESSION, normalization=normalization)
    COSMiK.extract(train)
    return COSMiK.brute_predict(test, 'default'), COSMiK.n_rules, sum([p is None for p in COSMiK.predict(test)])

def cream(model, train, test, normalization):
    CReEPy = Extractor.creepy(model, clustering=Clustering.cream, depth=5, error_threshold=.5, gauss_components=10,
                              output=Target.REGRESSION, normalization=normalization)
    CReEPy.extract(train)
    return CReEPy.brute_predict(test), CReEPy.n_rules, sum([p is None for p in CReEPy.predict(test)])

def exact(model, train, test, normalization):
    CReEPy = Extractor.creepy(model, clustering=Clustering.exact, depth=5, error_threshold=.5, gauss_components=10,
                              output=Target.REGRESSION, normalization=normalization)
    CReEPy.extract(train)
    return CReEPy.brute_predict(test), CReEPy.n_rules, sum([p is None for p in CReEPy.predict(test)])

In [7]:
bartels = pd.read_csv("data/bartels.csv", parse_dates = [1, 2])

extractors = ['GridEx', 'GridREx', 'CART', 'COSMiK', 'ExACT', 'CREAM']

TESTB = [i for i in range(2491, 2509)]

predicted = {name: [] for name in ['index', 'V', 'model'] + extractors}

rules = {name: [] for name in ['BR'] + extractors}

missed = {name: [] for name in ['BR'] + extractors}

for testB in TESTB[:3]:
    rules['BR'].append(testB)
    missed['BR'].append(testB)
    print(testB)

    data = pd.read_csv(f'data/pruned.csv', parse_dates=[0], index_col=0)
    train, test = getTrainTest(data, testB)

    predicted['index'] += list(test.index.values)
    predicted['V'] += list(test.V.values)

    scaler, scaledTrain, normalization = getScaler(train, f"test{testB}")
    scaledTest = pd.DataFrame(scaler.transform(test), columns=test.columns).iloc[:, :-1]
    m, s = normalization[test.columns[-1]]

    model = KNN(200, weights='distance', p=1).fit(scaledTrain.iloc[:, :-1], scaledTrain.iloc[:, -1])
    #dump(model, f"models/RF/{k}_{name}_{testB}.joblib")
    predicted['model'] += list(model.predict(scaledTest) * s + m)

    for name, fun in zip(extractors, [cart, cart, cart, cosmik, exact, cream]):
        print(name)
        #if name in ['GridREx', 'CART', 'COSMiK']:
        #    continue
        pred, n, miss = fun(model, scaledTrain, scaledTest, normalization)
        predicted[name] += list(pred)
        rules[name].append(n)
        missed[name].append(miss)

2491
GridEx
GridREx
CART
COSMiK
ExACT
CREAM
2492
GridEx
GridREx
CART
COSMiK
ExACT
CREAM
2493
GridEx
GridREx
CART
COSMiK
ExACT
CREAM


In [15]:
p = pd.DataFrame(predicted)
for e in extractors:
    print(abs(p.V - p[e]).mean(), abs(p.model - p[e]).mean())

b = 2493
#plot(b, p['index'], p.model, p.COSMiK)
#plot(b, p['index'], p.model, p.ExACT)
#plot(b, p['index'], p.model, p.CREAM)

pd.DataFrame(rules)

90.20644923758856 47.26039351140736
90.20644923758856 47.26039351140736
90.20644923758856 47.26039351140736
73.38650661174208 31.857409982273143
71.27591988138312 36.1748132343631
70.26005494494277 33.783890819165826


,BR,GridEx,GridREx,CART,COSMiK,ExACT,CREAM
0,2491,10,10,10,4,6,6
1,2492,10,10,10,8,6,6
2,2493,10,10,10,9,6,7


In [ ]:
pd.DataFrame(predicted).to_csv("results/pred1.csv")
pd.DataFrame(rules).to_csv('results/rules1.csv')
pd.DataFrame(missed).to_csv('results/missed1.csv')

In [ ]:
[
    ('half2', 1.0),
    ('Bmin', 0.9014926374248455),
    ('Bmedian', 0.5931779258652916),
    ('Bmax', 0.5580664110760513),

    ('trend', 0.441328536097858),
    ('max', 0.19391808653199277),
    ('median', 0.16157357528452515),

    ('halfB2', 0.06672545822303622),
    ('min', 0.042323799412967836),
    ('B', 0.014457395044254808),
    ('halfB1', 0.005159557201584083),
    ('half1', 0.0023286124765232786),
]

In [ ]:
import pandas as pd

p = pd.read_csv('results/cartpredpruned.csv')
p = p[p.columns[2:]]
for c in p.columns[2:]:
    p[c] = abs(p[c] - p.model)
p.describe().loc['mean']